# Hybrid quantum-classical auto encoder 

In [ ]:
"""
Sophie Choe sophchoe@pdx.edu
Portland State University
9/24/2021
"""

In [ ]:
"""
This is an implementation of the hybrid quantum-classical auto-encoder model outlined in "Continuous-Variable 
quantum neural networks" Physical Review Research 1, 033063, 2019. The encoder consists of six classical neural 
network layers and the decoder consists of three quantum neural network layers. Input vectors of length 3 are 
encoded in vectors of length 2, they are used as parameters of the Displacement gate paremeters, followed by the 
quantum decoder producing output vectors of length 3. 

The classical layers are implemented with Keras. The quantum layers are implemented with Pennylane. With the 
Pennylane qml.qnn.KerasLayer plug-in, the whole network is converted to a Keras model, and Keras loss function 
and optimizer are used for training.

The loss function in the paper requires state vectors and probability, however Pennylane measurement module does 
not support state vector retrieval (as of 9/24/2021). Hence mean squared error from Keras is used instead on the 
output vectors of probability measurement. To get vectors of length 3, the cutoff dimension parameter is set to 3. 

It reaches around 70% training accuracy and around 75% evaluation accuracy
"""

Dependencies: anaconda-navigator==1.10.0 keras-nightly==2.5.0.dev2021032900 PennyLane==0.17.0 StrawberryFields==0.18.0 tensorflow @ file:///Users/uwe/miniconda3/conda-bld/tensorflow-split_1618075966251/work/tensorflow_pkg/tensorflow-2.4.0-cp38-cp38-macosx_10_9_x86_64.whl

In [37]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pennylane as qml
from pennylane import numpy as np

In [38]:
# ===================================================================================
#                       Create Data
# ===================================================================================

num_train = 100
num_test = 20
len_vector = 3

np.random.seed(1)
x_train = np.random.rand(num_train, len_vector)
x_test = np.random.rand(num_test, len_vector)

In [54]:
# ===================================================================================
#                       Classical Layers using Keras Sequential
# ===================================================================================

# Define first layer, hidden layers, and output layer with the output of two neurons

keras.backend.set_floatx('float32')
model = tf.keras.Sequential(
                            [layers.Dense(5, input_shape=(3,), activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(2, activation ="elu")]
                            )

In [40]:
# ===================================================================================
#                                Quantum Iterative Layer
# ===================================================================================

# Initialize weights to pass through the q_layer function. Tensor of size: num_layers x num_parameters  
# Params for the Rotation gate, Squeezing gate, Displacement gate, and Kerr gate

def init_weights(layers, modes, active_sd=0.0001, passive_sd=0.1):
    
    r1_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    s_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    r2_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    dr_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    k_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)

    weights = tf.concat([r1_weights, s_weights, r2_weights, dr_weights, k_weights], axis=1)
    weights = tf.Variable(weights)

    return weights

# Construct a quantum layer with initialized weights as variables

def q_layer(v):
    qml.Rotation(v[0], wires=0)
    qml.Squeezing(v[1], 0.0, wires=0)
    qml.Rotation(v[2], wires=0)
    qml.Displacement(v[3], 0.0, wires=0)
    qml.Kerr(v[4], wires=0)

In [41]:
# ===================================================================================
#                                Quantum Circuit
# ===================================================================================

# Use the output of the classical layers to initialize the quantum layers with the Displacement gate
# Iterate the quantum layers

num_modes = 1
num_basis = 3

dev = qml.device("strawberryfields.fock", wires=num_modes, cutoff_dim=num_basis) # select a devide 

@qml.qnode(dev, interface = "tf")
def quantum_nn(inputs, var):
    
    # Inputs: the output of classical layers as displacement gate parameters
    qml.Displacement(inputs[0], inputs[1], wires=0)    

    # quantum layers
    for v in var:
        q_layer(v)

    return qml.probs(wires=0)

In [55]:
# ===================================================================================
#                             Hybrid Model
# ===================================================================================

"""
Add the quantum layer to the classical to create a hybrid model
    1. initialize weights for quantum layers
    2. create a dictionary of weight shape to pass as one of the variables to covert to keras layer
    3. convert the quantum layer to a Keras layer
    4. add to the classical sequential model
"""

num_layers = 4
num_modes = 1

weigths = init_weights(num_layers, num_modes)
shape_tup = weigths.shape
weight_shapes = {'var': shape_tup}
qlayer = qml.qnn.KerasLayer(quantum_nn, weight_shapes, output_dim=4)
model.add(qlayer)

In [56]:
# ===================================================================================
#                           Loss Function and Optimizer
# ===================================================================================

loss = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
#opt = tf.keras.optimizers.SGD(learning_rate=0.02)

model.compile(opt, loss=loss, metrics=["accuracy"])

In [57]:
# ===================================================================================
#                                    Training
# ===================================================================================

model.fit(x_train, 
          x_train,
          epochs=200,
          batch_size=24,
          shuffle=True,
          validation_data=(x_test, x_test))

Epoch 1/200
5/5 [==============================] - 5s 943ms/step - loss: 0.1973 - accuracy: 0.2583 - val_loss: 0.1727 - val_accuracy: 0.4500
Epoch 2/200
5/5 [==============================] - 5s 916ms/step - loss: 0.1741 - accuracy: 0.4097 - val_loss: 0.1522 - val_accuracy: 0.4500
Epoch 3/200
5/5 [==============================] - 5s 921ms/step - loss: 0.1578 - accuracy: 0.5850 - val_loss: 0.1393 - val_accuracy: 0.6000
Epoch 4/200
5/5 [==============================] - 5s 917ms/step - loss: 0.1451 - accuracy: 0.5738 - val_loss: 0.1300 - val_accuracy: 0.5500
Epoch 5/200
5/5 [==============================] - 5s 916ms/step - loss: 0.1381 - accuracy: 0.5365 - val_loss: 0.1231 - val_accuracy: 0.5000
Epoch 6/200
5/5 [==============================] - 5s 913ms/step - loss: 0.1292 - accuracy: 0.5500 - val_loss: 0.1194 - val_accuracy: 0.5500
Epoch 7/200
5/5 [==============================] - 5s 914ms/step - loss: 0.1162 - accuracy: 0.5505 - val_loss: 0.1171 - val_accuracy: 0.5500
Epoch 8/200
5

Epoch 59/200
5/5 [==============================] - 5s 962ms/step - loss: 0.0928 - accuracy: 0.6449 - val_loss: 0.0965 - val_accuracy: 0.7000
Epoch 60/200
5/5 [==============================] - 5s 959ms/step - loss: 0.0958 - accuracy: 0.6056 - val_loss: 0.0972 - val_accuracy: 0.7000
Epoch 61/200
5/5 [==============================] - 5s 967ms/step - loss: 0.0995 - accuracy: 0.6390 - val_loss: 0.0968 - val_accuracy: 0.7000
Epoch 62/200
5/5 [==============================] - 5s 980ms/step - loss: 0.0989 - accuracy: 0.6756 - val_loss: 0.0966 - val_accuracy: 0.7000
Epoch 63/200
5/5 [==============================] - 6s 1s/step - loss: 0.0909 - accuracy: 0.6621 - val_loss: 0.0972 - val_accuracy: 0.7000
Epoch 64/200
5/5 [==============================] - 6s 1s/step - loss: 0.0949 - accuracy: 0.6536 - val_loss: 0.0973 - val_accuracy: 0.7000
Epoch 65/200
5/5 [==============================] - 6s 1s/step - loss: 0.0950 - accuracy: 0.6278 - val_loss: 0.0971 - val_accuracy: 0.6500
Epoch 66/200
5/

Epoch 117/200
5/5 [==============================] - 5s 1s/step - loss: 0.0963 - accuracy: 0.6558 - val_loss: 0.0961 - val_accuracy: 0.6500
Epoch 118/200
5/5 [==============================] - 5s 992ms/step - loss: 0.0897 - accuracy: 0.6656 - val_loss: 0.0956 - val_accuracy: 0.6500
Epoch 119/200
5/5 [==============================] - 5s 999ms/step - loss: 0.0919 - accuracy: 0.6775 - val_loss: 0.0955 - val_accuracy: 0.6500
Epoch 120/200
5/5 [==============================] - 5s 954ms/step - loss: 0.0873 - accuracy: 0.6885 - val_loss: 0.0953 - val_accuracy: 0.6500
Epoch 121/200
5/5 [==============================] - 5s 956ms/step - loss: 0.0899 - accuracy: 0.6675 - val_loss: 0.0954 - val_accuracy: 0.6500
Epoch 122/200
5/5 [==============================] - 5s 960ms/step - loss: 0.0953 - accuracy: 0.6918 - val_loss: 0.0951 - val_accuracy: 0.7000
Epoch 123/200
5/5 [==============================] - 5s 962ms/step - loss: 0.0920 - accuracy: 0.6568 - val_loss: 0.0952 - val_accuracy: 0.7000
Ep

5/5 [==============================] - 5s 954ms/step - loss: 0.0927 - accuracy: 0.7242 - val_loss: 0.0953 - val_accuracy: 0.7500
Epoch 176/200
5/5 [==============================] - 5s 1s/step - loss: 0.0913 - accuracy: 0.7141 - val_loss: 0.0952 - val_accuracy: 0.7000
Epoch 177/200
5/5 [==============================] - 5s 973ms/step - loss: 0.0875 - accuracy: 0.7180 - val_loss: 0.0949 - val_accuracy: 0.7500
Epoch 178/200
5/5 [==============================] - 5s 962ms/step - loss: 0.0912 - accuracy: 0.7064 - val_loss: 0.0948 - val_accuracy: 0.7500
Epoch 179/200
5/5 [==============================] - 5s 956ms/step - loss: 0.0899 - accuracy: 0.7163 - val_loss: 0.0945 - val_accuracy: 0.7500
Epoch 180/200
5/5 [==============================] - 6s 1s/step - loss: 0.0979 - accuracy: 0.7134 - val_loss: 0.0945 - val_accuracy: 0.7500
Epoch 181/200
5/5 [==============================] - 5s 1s/step - loss: 0.0937 - accuracy: 0.7268 - val_loss: 0.0946 - val_accuracy: 0.7000
Epoch 182/200
5/5 [===

In [58]:
# ===================================================================================
#                                    Evalutation
# ===================================================================================

results = model.evaluate(x_test, x_test, batch_size=128)
print("test loss, test acc:", results)

1/1 [==============================] - 0s 146ms/step - loss: 0.0941 - accuracy: 0.7500
test loss, test acc: [0.09412109851837158, 0.75]


In [59]:
pip freeze

absl-py @ file:///opt/concourse/worker/volumes/live/7b8ad720-017e-41da-51b8-fb0f9b168ca6/volume/absl-py_1623867239512/work
aiohttp @ file:///opt/concourse/worker/volumes/live/c3cb6692-82cf-4cbe-4c9c-30944b7532e9/volume/aiohttp_1614360972016/work
anaconda-client @ file:///opt/concourse/worker/volumes/live/1ce006cc-6454-4d7d-4ed8-8b41941b64eb/volume/anaconda-client_1624480173054/work
anaconda-navigator==1.10.0
antlr4-python3-runtime==4.8
anyio @ file:///opt/concourse/worker/volumes/live/64740ac7-3a9c-4fbb-6685-a51c4ff8b4ca/volume/anyio_1617783319350/work/dist
appdirs==1.4.4
appnope @ file:///opt/concourse/worker/volumes/live/5f13e5b3-5355-4541-5fc3-f08850c73cf9/volume/appnope_1606859448618/work
argon2-cffi @ file:///opt/concourse/worker/volumes/live/d733ceb5-7f19-407b-7da7-a386540ab855/volume/argon2-cffi_1613037492998/work
astor==0.8.1
astunparse==1.6.3
async-generator @ file:///home/ktietz/src/ci/async_generator_1611927993394/work
async-timeout==3.0.1
attrs @ file:///tmp/build/80754af9/

Note: you may need to restart the kernel to use updated packages.
